In [2]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)

Loading custom CUDA kernels...
Could not load the custom kernel for multi-scale deformable attention: /root/.cache/torch_extensions/py310_cu118/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Using TensorFlow backend


Loading custom CUDA kernels...
Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu118/cuda_kernel...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=cuda_kernel -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/TH -isystem /opt/conda/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /opt/conda/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=1 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_75,code=compute_75 -gencode=arch=compute_75,code=sm_75 --compiler-options '-fPIC' -std=c++17 -c /opt/conda/lib/python3.10/site-packages/transformers/kernels/mra/cuda_kernel.cu -o cuda_kernel.cuda.o 
[2/4] c++ -MMD -MF torch_extension.o.d -DTORC

Loading extension module cuda_kernel...


TF version 2.12.0


In [5]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders

In [16]:
MAX_LEN = 192
PATH = '/kaggle/input/tf-roberta/'
tokenizer = Tokenizer(models.BPE(
    vocab_file=PATH+'vocab-roberta-base.json', 
    merges_file=PATH+'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
))
sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv').fillna('')
train.head()

Ignored unknown kwarg option vocab_file
Ignored unknown kwarg option merges_file
Ignored unknown kwarg option lowercase
Ignored unknown kwarg option add_prefix_space


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on the release...","Sons of ****,",negative


In [17]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [18]:
test = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

In [19]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
    tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x1 = tf.keras.layers.Conv1D(1,1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
    x2 = tf.keras.layers.Conv1D(1,1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [20]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [21]:
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

loading configuration file /kaggle/input/tf-roberta/config-roberta-base.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": null,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


#########################
### FOLD 1
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /kaggle/input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
687/687 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to v0-roberta-0.h5
687/687 [==============================] - 955s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 2/3
687/687 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 2: val_loss did not improve from 0.00000
687/687 [==============================] - 890s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 3/3
687/687 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss

loading configuration file /kaggle/input/tf-roberta/config-roberta-base.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": null,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


...

Epoch 1/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to v0-roberta-3.h5
688/688 [==============================] - 967s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 2/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 2: val_loss did not improve from 0.00000
688/688 [==============================] - 916s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 3/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss

loading configuration file /kaggle/input/tf-roberta/config-roberta-base.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": null,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


>>>> FOLD 4 Jaccard = 0.5784787933730345

#########################
### FOLD 5
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /kaggle/input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to v0-roberta-4.h5
688/688 [==============================] - 950s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 2/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00
Epoch 2: val_loss did not improve from 0.00000
688/688 [==============================] - 916s 1s/step - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss: 0.0000e+00 - val_loss: 0.0000e+00 - val_activation_loss: 0.0000e+00 - val_activation_1_loss: 0.0000e+00
Epoch 3/3
688/688 [==============================] - ETA: 0s - loss: 0.0000e+00 - activation_loss: 0.0000e+00 - activation_1_loss

In [22]:
all = []
for k in range(input_ids_t.shape[0]):
    a = np.argmax(preds_start[k,])
    b = np.argmax(preds_end[k,])
    if a>b: 
        st = test.loc[k,'text']
    else:
        text1 = " "+" ".join(test.loc[k,'text'].split())
        enc = tokenizer.encode(text1)
        st = tokenizer.decode(enc.ids[a-1:b])
    all.append(st)

In [23]:
test['selected_text'] = all
test[['textID','selected_text']].to_csv('submission.csv',index=False)
pd.set_option('max_colwidth', 60)
test.sample(25)

,textID,text,sentiment,selected_text
2307,cfd68d702e,I want it to be summer. NOW. why is it sprinkling STILL!?!?,negative,I want it to be summer. NOW. why is it sprinkling STILL!?!?
1355,7c229b34c1,_Itx no credits for me unfortunately!,negative,_Itx no credits for me unfortunately!
8,e64208b4ef,and within a short time of the last clue all of them,neutral,and within a short time of the last clue all of them
352,01fb29c047,"bored bored bored! wish i had something to do tomorrow, ...",negative,"bored bored bored! wish i had something to do tomorrow, ..."
3190,25694a4e38,JT should just be a regular on SNL,neutral,JT should just be a regular on SNL
2000,838c1184b4,Pissed that my bluetooth headset`s battery out lasted my...,negative,Pissed that my bluetooth headset`s battery out lasted my...
3519,91f8e27d4c,"Yes you are, thanks Haha you do? Field of flowers don`...",positive,
2879,65019c18cb,hey Sherri -- don`t give up b/c they`re married; they m...,positive,
1946,4331a46aba,awhile ago it freaking hot! now it`s wet turn off!,negative,awhile ago it freaking hot! now it`s wet turn off!
3295,08c5eb64a5,Sorry bro That`s rough.,negative,Sorry bro That`s rough.
